In [ ]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def cleandata(path,isdf):
    df = pd.read_feather(path)
    #Time
    df['date'] = pd.to_datetime(df.date, format='%Y%m%d')
    df['week'] = df.date.apply(lambda t: t.dayofweek)
    df['day'] = df.date.apply(lambda t: t.dayofyear)
    #drop repeated feature
    for i in df.columns:
        if sum(df[i] == df[i][0]) == df.shape[0]:
            df.drop([i], axis=1, inplace=True)
    #geoNetwork=
    #totals
    df.dropna(axis=0, inplace=True, subset=['totals_pageviews'])
    df['totals_bounces'] = df.totals_bounces.fillna(0).astype('float')
    df['totals_newVisits'] = df.totals_newVisits.fillna(0).astype('float')
    df['totals_hits'] = df.totals_hits.astype('float')
    df['totals_pageviews'] = df.totals_pageviews.astype('float')
    df['diff'] = df.totals_hits - df.totals_pageviews
    #trafficSource
    df['havegclid'] = df['trafficSource_adwordsClickInfo.gclId'].notnull()
    df['istruedirect'] = (df.trafficSource_isTrueDirect.notnull()) & (df.channelGrouping != 'Direct') 
    df['path'] = df.trafficSource_source +  df.trafficSource_referralPath.fillna('') 
    if isdf == True:
        df.drop(['trafficSource_campaignCode'],axis=1,inplace=True)
    else:
        pass
    #drop
    droplist = [
        'sessionId',
        'date',
        'device_isMobile',
        'visitStartTime',
        'geoNetwork_continent',
        'geoNetwork_region',
        'geoNetwork_metro',
        'geoNetwork_country',
        'trafficSource_adContent',
        'trafficSource_adwordsClickInfo.gclId',
        'trafficSource_medium',
        'trafficSource_adwordsClickInfo.isVideoAd',
        'trafficSource_adwordsClickInfo.adNetworkType',
        'trafficSource_adwordsClickInfo.page',
        'trafficSource_adwordsClickInfo.slot',
        'trafficSource_keyword',
        'trafficSource_campaign',
        'trafficSource_source',
        'trafficSource_referralPath',
        'trafficSource_isTrueDirect',
        'visitId',
        'fullVisitorId'
    ]
    df.drop(droplist, axis=1, inplace=True)
    return df
    

In [ ]:
trainpath = '/home/leechh/桌面/tempfile/R/train.feather'

train = cleandata(trainpath,True)

In [ ]:
testpath = '/home/leechh/桌面/tempfile/R/test.feather'

test = cleandata(testpath,False)

In [ ]:
idx = train.totals_transactionRevenue.notnull()
train = train[idx].reset_index()

In [ ]:
train.head()

In [ ]:
#device
convertlist = ['device_browser', 'device_operatingSystem','geoNetwork_subContinent']
for i in convertlist:
    idx1 = train[i].value_counts().index[:10].values
    idx2 = test[i].value_counts().index[:10].values
    whitelist = set(list(idx1)+list(idx2))
    traintotal = set(list(train[i].unique())+['Notin'])
    testtotal = set(list(test[i].unique())+['Notin'])
    trainlist  = traintotal.difference(whitelist)
    testlist = testtotal.difference(whitelist)
    train[i].replace(list(trainlist),'other',inplace=True)
    test[i].replace(list(testlist),'other',inplace=True)

In [ ]:
#device
convertlist = ['geoNetwork_city','geoNetwork_networkDomain','path']
for i in convertlist:
    idx1 = train[i].value_counts().index[:20].values
    idx2 = test[i].value_counts().index[:20].values
    whitelist = set(list(idx1)+list(idx2))
    traintotal = set(list(train[i].unique())+['Notin'])
    testtotal = set(list(test[i].unique())+['Notin'])
    trainlist  = traintotal.difference(whitelist)
    testlist = testtotal.difference(whitelist)
    train[i].replace(list(trainlist),'other',inplace=True)
    test[i].replace(list(testlist),'other',inplace=True)

In [ ]:
target = train.totals_transactionRevenue
train.drop(['totals_transactionRevenue','index'], axis=1, inplace=True)

In [ ]:
realtarget = target.astype('float')/(10**6)
logtarget = np.log1p(target.astype('float')+1)

In [ ]:
train.head()

In [ ]:
lelist = ['channelGrouping',
          'device_browser',
          'device_deviceCategory',
          'device_operatingSystem',
          'geoNetwork_city',
          'geoNetwork_networkDomain',
          'geoNetwork_subContinent',
          'path'
         ]

for col in lelist:
    le = LabelEncoder()
    fitlabel = le.fit(list(train[col].values)+list(test[col].values))
    train[col] = le.transform(list(train[col].values))
    test[col] = le.transform(list(test[col].values))

In [ ]:
train.head()

In [ ]:
trainarray = train[lelist].values
testarray = test[lelist].values
trainlist = []
testlist = []
for i in np.arange(trainarray.shape[1]):
    ohe = OneHotEncoder()
    ohe.fit(np.append(trainarray[:,i], testarray[:,i])[:,np.newaxis])
    trainlist.append(ohe.transform(trainarray[:,i][:,np.newaxis]).toarray())
    testlist.append(ohe.transform(testarray[:,i][:,np.newaxis]).toarray())
    
x_train = np.hstack(trainlist)
x_test = np.hstack(testlist)

In [ ]:
columns = list(set(train.columns).difference(lelist))

x_train = np.hstack((train[columns].values,x_train))
x_test = np.hstack((test[columns].values,x_test))

In [ ]:
np.save('/home/leechh/tempfile/R/x_train1030.npy',x_train.astype('float'))
np.save('/home/leechh/tempfile/R/x_test1030.npy',x_test.astype('float'))

In [ ]:
np.save('/home/leechh/tempfile/R/target.npy',target.astype('float'))
np.save('/home/leechh/tempfile/R/logtarget.npy',logtarget.astype('float'))
np.save('/home/leechh/tempfile/R/realtarget.npy',realtarget.astype('float'))

In [ ]:
gc.collect()